In [ ]:
# === Environment Setup ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
try:
    import pandas_datareader.data as web
    PDR_AVAILABLE = True
except ImportError:
    PDR_AVAILABLE = False
from IPython.display import display, Markdown

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 14, 'figure.figsize': (12, 8), 'figure.dpi': 150})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Utility Functions ---
def note(msg): display(Markdown(f"<div class='alert alert-info'>📝 {msg}</div>"))
def sec(title): print(f'\n{80*"="}\n| {title.upper()} |\n{80*"="}')

note("Environment initialized for Vector Autoregression (VAR) Models.")

# Chapter 8.4: Vector Autoregression (VAR) Models

---

### Table of Contents

1.  [**Introduction: From Univariate to Multivariate Time Series**](#intro)
2.  [**The VAR(p) Model**](#var-model)
    - [Estimation and Lag Selection](#estimation)
3.  [**Granger Causality**](#granger)
4.  [**Impulse Response Functions (IRFs)**](#irf)
    - [The Identification Problem](#identification)
5.  [**Case Study: A Macroeconomic VAR for the US Economy**](#case-study)
6.  [**Exercises**](#exercises)
7.  [**Summary and Key Takeaways**](#summary)

<a id='intro'></a>
## 1. Introduction: From Univariate to Multivariate Time Series

The ARIMA models we have studied so far are **univariate**—they model a single time series based only on its own past. However, economic variables are interconnected. GDP growth is affected by monetary policy, inflation depends on unemployment, and so on. To model these rich dynamic relationships, we need a **multivariate** framework.

Prior to the 1980s, macroeconomic modeling was dominated by large-scale structural equation models. These models were heavily based on economic theory but were criticized by **Robert Lucas** in what became known as the **Lucas critique**. He argued that the parameters of these models were not stable under policy changes, as they failed to account for how rational agents would change their expectations and behavior. 

**Vector Autoregression (VAR)** models, introduced in a seminal 1980 paper by **Christopher Sims** (who later won a Nobel Prize for this work), were proposed as a direct response to this critique. A VAR is a more flexible, data-driven approach that imposes fewer theoretical restrictions. It treats every variable in the system as endogenous, creating a system of equations where each variable is regressed on its own lags and the lags of all other variables. This allows us to capture the complex feedback loops present in the economy and, crucially, to trace out how shocks to one variable dynamically affect all other variables in the system, with more transparent identifying assumptions.

<a id='var-model'></a>
## 2. The VAR(p) Model

A VAR(p) model with $k$ variables is a system of $k$ equations. For a simple two-variable ($y_1, y_2$) VAR(1) model, the system would be:
$$ y_{1,t} = c_1 + \phi_{11,1} y_{1,t-1} + \phi_{12,1} y_{2,t-1} + \epsilon_{1,t} $$ 
$$ y_{2,t} = c_2 + \phi_{21,1} y_{1,t-1} + \phi_{22,1} y_{2,t-1} + \epsilon_{2,t} $$ 
Here, the current value of $y_1$ depends on the first lag of both $y_1$ and $y_2$. The error terms $(\epsilon_{1,t}, \epsilon_{2,t})$ are assumed to be white noise, but they can be contemporaneously correlated with each other, $Cov(\epsilon_{1,t}, \epsilon_{2,t}) = \sigma_{12}$.

<a id='estimation'></a>
### Estimation and Lag Selection
Since the right-hand-side variables in each equation are all lagged (pre-determined), there are no endogeneity issues, and each equation can be estimated individually using OLS. The main specification choice is the **lag order (p)**. This is almost always chosen by estimating the VAR for a range of different lag lengths and selecting the one that minimizes an information criterion like the AIC or BIC.

<a id='granger'></a>
## 3. Granger Causality

A key question in a VAR is whether one variable is useful for forecasting another. The **Granger causality test** provides a statistical answer to this. The principle is:

> Variable $X$ is said to **Granger-cause** variable $Y$ if past values of $X$ contain information that helps predict future values of $Y$, even after accounting for the information contained in past values of $Y$ itself.

In the context of our two-variable VAR(1) above, we would test if $y_2$ Granger-causes $y_1$ by performing an F-test on the null hypothesis that the coefficient $\phi_{12,1}$ is zero. If we reject the null, it means that past values of $y_2$ have statistically significant predictive power for $y_1$.

<a id='irf'></a>
## 4. Impulse Response Functions (IRFs)

The primary tool for analyzing a VAR is the **Impulse Response Function (IRF)**. An IRF traces the dynamic effect of a one-time shock (an "impulse") to one of the variables on all other variables in the system over time.

<a id='identification'></a>
### The Identification Problem
A major challenge is that the raw error terms from the VAR estimation (the *reduced-form residuals*, $\epsilon_t$) are often correlated with each other. For example, $Cov(\epsilon_{gdp,t}, \epsilon_{inf,t}) \neq 0$. This makes it hard to interpret a shock to one variable as a pure, isolated event. Is an increase in $\epsilon_{gdp,t}$ a 'supply shock' or is it just statistically correlated with an inflationary 'demand shock' that also happened to occur?

To generate economically meaningful IRFs, we must impose **identification** assumptions to transform the correlated reduced-form errors into a set of uncorrelated **structural shocks**, $u_t$. The goal is to find a mapping from the unobserved structural shocks to the observed residuals. The diagram below illustrates this conceptual process.

![VAR Identification Diagram](../images/png/var_identification_diagram.png)

The most common identification scheme, and the default in `statsmodels`, is the **Cholesky decomposition** of the covariance matrix of the errors, $\Sigma_\epsilon$. This imposes a recursive causal ordering. For a system with variables $[y_1, y_2, y_3]$, it assumes that a structural shock to $y_1$ can affect all variables contemporaneously. A shock to $y_2$ can affect $y_2$ and $y_3$ contemporaneously, but only affects $y_1$ with a lag. A shock to $y_3$ only affects itself contemporaneously and all other variables with a lag. The choice of this ordering is a strong economic assumption that must be justified by theory.

In [ ]:
<a id='case-study'></a>
sec("Case Study: A Simple Macroeconomic VAR")

# 1. Load and prepare data
series_to_load = {
    'GDPC1': 'GDP_Growth',
    'CPIAUCSL': 'Inflation',
    'FEDFUNDS': 'Fed_Funds_Rate'
}
if PDR_AVAILABLE:
    note("Attempting to download quarterly US macro data from FRED.")
    try:
        start = '1960-01-01'
        end = '2019-12-31' # End before COVID for a more stable period
        macro_data_raw = web.DataReader(list(series_to_load.keys()), 'fred', start, end)
        note("Data downloaded successfully.")
    except Exception as e:
        note(f"Could not download data from FRED ({e}). Falling back to local CSVs.")
        gdp = pd.read_csv('data/GDPC1.csv', index_col='observation_date', parse_dates=True)
        cpi = pd.read_csv('data/CPIAUCSL.csv', index_col='observation_date', parse_dates=True)
        fedfunds = pd.read_csv('data/FEDFUNDS.csv', index_col='observation_date', parse_dates=True)
        macro_data_raw = pd.concat([gdp, cpi, fedfunds], axis=1)
else:
    note("pandas_datareader not available. Loading data from local CSVs.")
    gdp = pd.read_csv('data/GDPC1.csv', index_col='observation_date', parse_dates=True)
    cpi = pd.read_csv('data/CPIAUCSL.csv', index_col='observation_date', parse_dates=True)
    fedfunds = pd.read_csv('data/FEDFUNDS.csv', index_col='observation_date', parse_dates=True)
    macro_data_raw = pd.concat([gdp, cpi, fedfunds], axis=1)

# Process the data
macro_data = pd.DataFrame()
macro_data['GDP_Growth'] = macro_data_raw['GDPC1'].pct_change(4).dropna() * 100
macro_data['Inflation'] = macro_data_raw['CPIAUCSL'].pct_change(12).dropna() * 100
macro_data['Fed_Funds_Rate'] = macro_data_raw['FEDFUNDS']
macro_data = macro_data.resample('QS').mean().dropna()
macro_data = macro_data.loc['1980-01-01':'2019-12-31']

# A Note on Stationarity and Cointegration
note("Most macroeconomic time series are non-stationary (I(1)). If a linear combination of I(1) variables is stationary (I(0)), they are said to be **cointegrated**. In this case, the appropriate model is a **Vector Error Correction Model (VECM)**, which is a VAR in first-differences that includes a term to account for the long-run cointegrating relationship. For pedagogical simplicity, we will estimate a standard VAR on the first-differenced data, which is appropriate if the variables are not cointegrated. A full analysis would require formal tests for cointegration.")

from statsmodels.tsa.stattools import adfuller
for name in macro_data.columns:
    p_val = adfuller(macro_data[name])[1]
    print(f'ADF p-value for {name}: {p_val:.4f}')
note("As expected, the series appear non-stationary. We will proceed with a VAR on the first differences.")
data_diff = macro_data.diff().dropna()

# 2. Fit the VAR model
model = VAR(data_diff)
lag_order_results = model.select_order(maxlags=10)
selected_lag = lag_order_results.bic
note(f"BIC selects a lag order of p = {selected_lag}")

results = model.fit(selected_lag)
# print(results.summary())

# 3. Granger Causality Test
note("Testing if the Fed Funds Rate Granger-causes GDP Growth.")
gc_test = results.test_causality('GDP_Growth', ['Fed_Funds_Rate'], kind='f')
print(gc_test)

# 4. Impulse Response Functions
note("Plotting Impulse Response Functions to a shock in the Fed Funds Rate.")
irf = results.irf(16) # IRF for 16 quarters
irf.plot(orth=True)
plt.suptitle('Impulse Responses to Structural Shocks (Cholesky Order)', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()
note("The plots in the first column show the response of each variable to a one-standard-deviation shock to GDP Growth. The third column shows the response to a monetary policy shock (a shock to the Fed Funds Rate). As expected, a positive shock to the Fed Funds Rate leads to a decrease in GDP Growth and Inflation.")

### 4.1 Forecast Error Variance Decomposition (FEVD)
Another key tool for VAR analysis is the **Forecast Error Variance Decomposition (FEVD)**. It shows the proportion of the forecast error variance for each variable that is attributable to shocks from itself and from the other variables in the system, at different forecast horizons.

For example, the FEVD for GDP growth might show that at a 1-quarter horizon, 90% of its forecast error variance is due to its own shocks, but at a 20-quarter horizon, monetary policy shocks account for 30% of the variance. This helps us assess the relative importance of different shocks in driving the fluctuations of each variable.

In [ ]:
sec("Forecast Error Variance Decomposition")

note("Plotting the FEVD for a 20-quarter horizon.")
fevd = results.fevd(20)
fevd.plot()
plt.suptitle('Forecast Error Variance Decomposition', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()
note("The plots show how the contribution of different shocks to the forecast error variance of each variable evolves over time. For example, in the 'Fed_Funds_Rate' plot, we see that shocks to GDP and Inflation become more important for explaining its variance at longer horizons.")

<a id='exercises'></a>\n## 6. Exercises\n\n1.  **Interpreting VAR Output:** Look at the summary of the estimated VAR model (uncomment the print statement). Find the equation for `GDP_Growth`. Which variables have statistically significant coefficients for predicting GDP growth?\n2.  **Structural Shocks:** The IRFs plotted above use a Cholesky decomposition for identification. The default ordering is the order of the variables in the dataset: `[GDP_Growth, Inflation, Fed_Funds_Rate]`. What economic assumption is this ordering making about the contemporaneous relationships between the variables?\n3.  **Alternative Ordering:** How might you change the ordering of the variables if you believed that the Federal Reserve sets its interest rate based on the *current* quarter's inflation and GDP data, but that its policy only affects GDP and inflation with a lag?\n4.  **Interpreting FEVD:** Look at the FEVD plot for `GDP_Growth`. What is the approximate percentage of its forecast error variance at the 20-quarter horizon that is explained by `Fed_Funds_Rate` shocks? What does this imply about the importance of monetary policy for business cycle fluctuations in this model?

<a id='summary'></a>\n## 7. Summary and Key Takeaways\n\nThis chapter introduced Vector Autoregression (VAR), the workhorse model for multivariate time series analysis in modern macroeconomics.\n\n**Key Concepts**:\n- **Multivariate Framework**: VARs model multiple endogenous variables together in a system, capturing the rich feedback loops and dynamic interdependencies common in economic data.\n- **Estimation**: VAR models are simple to estimate, as each equation can be handled with OLS. The key specification choice is the lag length, which is typically chosen using information criteria.\n- **Granger Causality**: A statistical test to determine if past values of one variable are useful in forecasting another.\n- **Impulse Response Functions (IRFs)**: The primary tool for analysis. IRFs trace out the dynamic response of all variables in the system to a shock in one variable. They are essential for understanding the transmission mechanisms of economic shocks.\n- **Identification**: To give IRFs a causal interpretation, we must impose identification assumptions (e.g., a Cholesky decomposition) to transform the correlated model errors into uncorrelated, economically meaningful structural shocks.

### Solutions to Exercises\n\n---\n\n**1. Interpreting VAR Output:**\nYou would look at the `p-values` (P>|z|) for the coefficients in the `GDP_Growth` equation. You would likely find that past lags of GDP Growth itself are significant, and past lags of the other variables may or may not be, depending on the lag order chosen.\n\n---\n\n**2. Structural Shocks:**\nThe Cholesky ordering `[GDP_Growth, Inflation, Fed_Funds_Rate]` assumes a recursive structure. It assumes that a shock to GDP in the current quarter can contemporaneously affect inflation and the Fed Funds Rate. It assumes a shock to inflation can affect the Fed Funds Rate contemporaneously, but can only affect GDP with a lag. Finally, it assumes that a shock to the Fed Funds Rate can only affect GDP and inflation with a lag. This is a strong and often debated assumption.\n\n---\n\n**3. Alternative Ordering:**\nIf you believe the Fed responds to current economic conditions, you would place the `Fed_Funds_Rate` last in the ordering. This would mean that shocks to GDP and inflation can contemporaneously affect the interest rate. A common ordering based on this logic is `[GDP_Growth, Inflation, Fed_Funds_Rate]`, which is the default we used. If you believed GDP was the slowest-moving variable, you might use `[Inflation, Fed_Funds_Rate, GDP_Growth]`. The choice of ordering is a key part of structural VAR analysis.\n\n---\n\n**4. FEVD:**\nThe Forecast Error Variance Decomposition (FEVD) breaks down the variance of the forecast error for each variable into the parts attributable to shocks from each of the other variables in the system. For example, it might show that 60% of the variance in the 8-quarter-ahead forecast error for GDP is due to its own past shocks, 30% is due to monetary policy shocks, and 10% is due to inflation shocks. It is a useful tool for assessing the relative importance of different shocks in driving the fluctuations of the variables in the system.